# Practice session 5: section 6

# Background

Integration can be performed on functions that we know, but also on functions that we don't know. One such function can describe the shape of a polygon such as the cross-section of a river. 

# Problem

## Background

When estimating such a polygon we will have measurements of the depth of the river taken at regular intervals.

## Data

#### Exercise

The depths of a river H are measured at equally spaced distances across a channel as tabulated below. The river’s crosssectional area can be determined by integration as in

$$ \int_0^xH(x)dx$$

```
x, m |  0  2    4  6  8    10   12    14    16
H, m |  0  1.9  2  2  2.4  2.6  2.25  1.12  0
```



## Tasks

* Determine the cross-sectional area, both by the trapezoidal rule and by the 1/3 Simpson's rule with multiple application.

* Fit cuadratic splines to the points, and then calculate the area under that curve with multiple application, considering 25 segments. 

    * Hint: when you build a big sparse matrix, you don't need to write the zeroes. Just use `np.zeroes()` and fill in the non-zero values. If you're feeling adventurous, you can check the [documentation for sparse matrices in scipy]. 
    * Notice that when you build the matrix, you will have to arrange the rows in a certain way to avoid zeroes on the diagonal. It might be worth it to write a function to build the matrix, since it's pretty big.

* Compare both measurements with the Python integration of the estimated section.

* Plot both the measurements and your estimation of the shape of the cross-section. 

[documentation for sparse matrices in scipy]: https://docs.scipy.org/doc/scipy/reference/sparse.html

## Solution



In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

xs = np.array([0, 2, 4, 6, 8, 10, 12, 15, 16])
hs = np.array([0, 1.9, 2, 2, 2.4, 2.6, 2.25, 1.12, 0])

### Trapezoidal rule

In [7]:
area = 0

for segment in range(len(xs) - 1):
    a = xs[segment]
    b = xs[segment + 1]
    ha = hs[segment]
    hb = hs[segment + 1]
    
    
    area += (b-a)*(hb + ha)/2
    
area

29.665000000000003

### 1/3 Simpson's rule

The equation is:

$$\int_a^b f(x)dx \approx \int_a^b f_2(x)dx \approx \frac{b-a}{6}[f(x_0) + 4f(x_1) +f(x_2)]$$

We need three points for each segment, so we will only be able to get 4 segments.

In [24]:
area = 0

for segment in list(range(0,len(xs) - 1,2)):
    a = xs[segment]
    b = xs[segment + 2]
    hx0 = hs[segment]
    hx1 = hs[segment + 1]
    hx2 = hs[segment + 2]
    
    
    area += (b-a)*(hx0 + 4*hx1 + hx2)/6
    
area

29.18666666666667

### Fit quadratic splines

We have 9 points, so we'll fit 8 second-degree functions. Remember:

* The function values of adjacent polynomials must be equal at the interior knots.
* The first and last functions must pass through the end points.
* The first derivatives at the interior knots must be equal.
* Assume that the second derivative is zero at the first point

All of that should add to 24 equations.

In [106]:
AB = np.zeros((24,25))

# f1'' at x=0
AB[0,0] = 2

# f1 at x=0
AB[1, :3] = [0,0,1]
AB[1,24] = 0

# f1 at x=2
AB[2, :3] = [4,2,1]
AB[2,24] = 1.9

# f1' and f2' at x=2
AB[3, :2] = [4,1]
AB[3, 3:5] = [-4,-1]

# f2 at x=2
AB[4, 3:6] = [4,2,1]
AB[4, 24] = 1.9

# f2 at x=4
AB[5, 3:6] = [16,4,1]
AB[5, 24] = 2

# f2' and f3' at x=4
AB[6, 3:5] = [8,1]
AB[6, 6:8] = [-8,-1]
 
# f3 at x=4
AB[7, 6:9] = [16, 4, 1]
AB[7, 24] = 2

# f3 at x=6
AB[8, 6:9] = [16, 4, 1]    
AB[8, 24] = 2

# f3' and f4; at x=6
AB[9, 6:8] = [12, 1]
AB[9, 9:11] = [-12, -1]

# f4 at x=6
AB[10, 9:12] = [36,6,1]
AB[10, 24] = 2

# f4 at x=8
AB[11, 9:12] = [64,8,1]
AB[11, 24] = 2.4

# f4' and f5' at x=8
AB[12, 9:11] = [16, 1]
AB[12, 12:14] = [-16, -1]

# f5 at x=8
AB[13, 12:15] = [64, 8, 1]
AB[13, 24] = 2.4

# f5 at x=10
AB[14, 12:15] = [100, 10, 1]
AB[14, 24] = 2.6

# f5' and f6' at x=10
AB[15, 12:14] = [20, 1]
AB[15, 15:17] = [-20, -1]

# f6 at x=10
AB[16, 15:18] = [100, 10, 1]
AB[16, 24] = 2.6

# f6 at x=12
AB[17, 15:18] = [144, 12, 1]
AB[17, 24] = 2.25

# f6' and f7' at x=12
AB[18, 15:17] = [24, 1]
AB[18, 18:20] = [-24,-1]

# f7 at x=12
AB[19, 18:21] = [144, 12, 1]
AB[19, 24] = 2.25

# f7 at x=14
AB[20, 18:21] = [176, 14, 1]
AB[20, 24] = 1.12

# f7' and f8' at x=14
AB[21, 18:20] = [28, 1]
AB[21, 21:23] = [-28,-1]

# f8 at x=14
AB[22, 21:24] = [176,14,1]
AB[22, 24] = 1.12

# f8 at x=16
AB[23, 21:24] = [256, 16, 1]
AB[23, 24] = 0

AB[:10,:10]

array([[  2.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  4.,   2.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  4.,   1.,   0.,  -4.,  -1.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   4.,   2.,   1.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,  16.,   4.,   1.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   8.,   1.,   0.,  -8.,  -1.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,  16.,   4.,   1.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,  16.,   4.,   1.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,  12.,   1.,   0., -12.]])

In [111]:
AB.diagonal()

array([  2.,   0.,   1.,  -4.,   2.,   1.,  -8.,   4.,   1., -12.,   6.,
         1., -16.,   8.,   1., -20.,  10.,   1., -24.,  12.,   1., -28.,
        14.,   1.])

In [112]:
AB[:,24]

array([0.  , 0.  , 1.9 , 0.  , 1.9 , 2.  , 0.  , 2.  , 2.  , 0.  , 2.  ,
       2.4 , 0.  , 2.4 , 2.6 , 0.  , 2.6 , 2.25, 0.  , 2.25, 1.12, 0.  ,
       1.12, 0.  ])

In [113]:
la.det(AB[:,:24])

0.0

In [114]:
la.solve(AB[:,:-1], AB[:,-1])

LinAlgError: Singular matrix